<a href="https://colab.research.google.com/github/Vedauvoo/TPQ-CPF/blob/main/EB_Week07_Backtester_OOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Checking python version Colab is using
!python --version

Python 3.10.12


In [4]:
#Checking scikit-learn version that Colab is using
import sklearn
print("Scikit-learn version:", sklearn.__version__)

Scikit-learn version: 1.3.2


In [5]:
#Updating the verion of scikit-learn to the latest version
#This line MUST be executed to ensure reproducable results
!pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 54.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [1]:
#Checking to make sure install of scikit-learn was successful & on latest version
#IMPORTANT:  MUST SELECT "RESTART SESSION" from "Runtime" drop down menu to get scikit learn version to update
import sklearn
print("Scikit-learn version:", sklearn.__version__)

Scikit-learn version: 1.5.1


In [ ]:
#Code to update colab to use the latest version of python; not needed at this time
#!sudo update-alternatives --config python3
#!sudo apt-get update -y
#!sudo apt-get install python3.12 python3.12-dev python3.12-distutils libpython3.12-dev
#!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
#!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 2
#!sudo update-alternatives --config python3
#!python --version

In [2]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
%matplotlib inline

In [3]:
class FinancialData(object):
    def __init__(self, symbol):
        self.symbol = symbol
        self.retrieve_data()

    def retrieve_data(self):
        self.raw = pd.read_csv('http://hilpisch.com/ref_eikon_eod_data.csv',
                                 index_col=0, parse_dates=True)
        #print("Original Length of raw = ",len(self.raw))
        self.data = pd.DataFrame(self.raw[self.symbol])
        #self.data['Returns'] = np.log(self.data / self.data.shift(1))
        self.data.dropna(inplace=True)
        #print("Original Length of data = ",len(self.data))

    def plot_data(self, cols=None):
        if cols is None:
            cols = [self.symbol]
        self.data[cols].plot(figsize=(10, 6), title=self.symbol)

In [4]:
class BacktestingBase(FinancialData):
    def __init__(self, symbol, amount, ptc):
        super(BacktestingBase, self).__init__(symbol)
        self.amount = amount  # cash balance
        self.initial_amount = amount  # initial cash position
        self.ptc = ptc #proportional transaction cost
        self.position = 0  # initial position is neutral
        self.trades = 0  # total number of individual trades
        self.rt_trades = 0 # number of round-trip trades (buy+sell)
        self.units = 0  # number of units of the instrument traded
        self.buy_price = 0 #Initial buy price for Long Entry
        self.sell_price = 0 #Initial sell price for Short Entry
        self.win_count = 0 #Number of trades that are winners
        self.win_rate = 0 #Number of winning trades ratioed to total # of trades
        self.cum_gain = 0 #Cumulative dollar gains
        #print("PTC = ", self.ptc)

    def get_date_price(self, bar):
        return str(self.test.index[bar])[:10], self.test[self.symbol].iloc[bar]

    def print_current_balance(self, bar):
        date, price = self.get_date_price(bar)
        #print('%s | current cash balance is   %8.2f' % (date, self.amount))

    def place_buy_order(self, bar, position=None):
        #print("In Buy Order Method: Position = ",position)
        date, price = self.get_date_price(bar)
        if self.initial_amount is not None and self.trades == 0:
            self.trades += 1
            #print("Trades = 0 & initial_amount not none. Computing units to buy")
            self.units = math.floor(self.initial_amount / (price * (1 + self.ptc)))
            self.buy_price = price*(1 + self.ptc)
            print(55*'=')
            print("%s | Trade # %d Opening LONG Trade" % (date, self.trades))
            print("%s | Trade # %d Amount = $%8.2f"% (date, self.trades,self.amount))
            print("%s | Trade # %d Buy Price w/o TCs = $%8.2f"% (date, self.trades,price))
            print("%s | Trade # %d Buy Price w/ TCs = $%8.2f"% (date, self.trades,self.buy_price))
            print("%s | Trade # %d # of Units = %8.2f"% (date, self.trades,self.units))
            self.amount -= self.units * (price * (1 + self.ptc))
            self.print_current_balance(bar)
        elif position == 0 and self.trades > 0:
            self.trades += 1
            #print("Position= 0 & # of trades > 0. Computing Units to buy")
            self.units = math.floor(self.amount / (price * (1 + self.ptc)))
            self.buy_price = price*(1 + self.ptc)
            print(55*'=')
            print("%s | Trade # %d Opening LONG Trade" % (date, self.trades))
            print("%s | Trade # %d Amount = $%8.2f"% (date, self.trades,self.amount))
            print("%s | Trade # %d Buy Price w/o TCs = $%8.2f"% (date, self.trades,price))
            print("%s | Trade # %d Buy Price w/ TCs = $%8.2f"% (date, self.trades,self.buy_price))
            print("%s | Trade # %d # of Units = %8.2f"% (date, self.trades,self.units))
            #print("computing amount in buy method")
            #Include proportional transaction cost
            self.amount -= self.units * (price * (1 + self.ptc))
            #print('%s | buying  %d units at price %8.2f' % (date, self.units, price))
            self.print_current_balance(bar)
        elif position == -1:
            self.buy_price = price*(1 + self.ptc)
            self.amount -= self.units * (price * (1 + self.ptc))
            self.trades += 1
            self.win_count += np.where(self.sell_price > price, 1, 0)
            print("%s | Trade # %d Closing SHORT Trade" % (date, self.trades))
            print("%s | Trade # %d Buy Price w/o TCs = $%8.2f"% (date, self.trades,price))
            print("%s | Trade # %d Buy Price w/ TCs = $%8.2f"% (date, self.trades,self.buy_price))
            print("%s | Trade # %d # of Units = %8.2f"% (date, self.trades,self.units))
            print("%s | Trade # %d Amount = %8.2f" % (date, self.trades,self.amount))
            print('%s | Trade # %d Gain/Loss %8.2f%%' % (date, self.trades, ((self.sell_price-(self.buy_price))/self.sell_price)*100))
            print('%s | Trade # %d Dollar Gain/Loss $%8.2f' % (date, self.trades, self.units*(self.sell_price-self.buy_price)))
            self.cum_gain += self.units*(self.sell_price-self.buy_price)
            print('%s | Trade # %d Cummulative Gain/Loss $%8.2f' % (date, self.trades,self.cum_gain))

    def place_sell_order(self, bar, position=None):
        date, price = self.get_date_price(bar)
        #print("In Sell Order Method: Position = ",position)
        if self.initial_amount is not None and self.trades == 0:
            self.trades += 1
            #print("Trades = 0 & initial_amount not none. Computing units to sell")
            self.units = math.floor(self.initial_amount / (price * (1 + self.ptc)))
            self.sell_price = price*(1 - self.ptc)
            print(55*'=')
            print("%s | Trade # %d Opening SHORT Trade" % (date, self.trades))
            print("%s | Trade # %d Amount = $%8.2f"% (date, self.trades,self.amount))
            print("%s | Trade # %d Sell Price w/o TCs = $%8.2f"% (date, self.trades,price))
            print("%s | Trade # %d Sell Price w/ TCs = $%8.2f"% (date, self.trades,self.sell_price))
            print("%s | Trade # %d # of Units = %8.2f"% (date, self.trades,self.units))
            self.amount += self.units * (price * (1 - self.ptc))
            self.print_current_balance(bar)
        elif position == 0 and self.trades > 0:
            self.trades += 1
            #print("Position=0 & # of trades > 0. Computing Units to sell")
            self.units = math.floor(self.amount / (price * (1 + self.ptc)))
            self.sell_price = price*(1 - self.ptc)
            print(55*'=')
            print("%s | Trade # %d Opening SHORT Trade" % (date, self.trades))
            print("%s | Trade # %d Amount = $%8.2f"% (date, self.trades,self.amount))
            print("%s | Trade # %d Sell Price w/o TCs = $%8.2f"% (date, self.trades,price))
            print("%s | Trade # %d Sell Price w/ TCs = $%8.2f"% (date, self.trades,self.sell_price))
            print("%s | Trade # %d # of Units = %8.2f"% (date, self.trades,self.units))
            self.amount += self.units * (price * (1 - self.ptc))
            #print('%s | selling %d units at price %8.2f' % (date, self.units, price))
            self.print_current_balance(bar)
        elif position == 1:
            self.sell_price = price*(1 - self.ptc)
            self.amount += self.units * (price * (1 - self.ptc))
            self.trades += 1
            self.win_count += np.where(self.buy_price < price, 1, 0)
            print("%s | Trade # %d Closing LONG Trade" % (date, self.trades))
            print("%s | Trade # %d Sell Price w/o TCs = $%8.2f"% (date, self.trades,price))
            print("%s | Trade # %d Sell Price w/ TCs = $%8.2f"% (date, self.trades,self.sell_price))
            print("%s | Trade # %d # of Units = %8.2f"% (date, self.trades,self.units))
            print("%s | Trade # %d Amount = %8.2f" % (date, self.trades,self.amount))
            print('%s | Trade # %d Gain/Loss %8.2f%%' % (date, self.trades, ((self.sell_price-self.buy_price)/self.buy_price)*100))
            print('%s | Trade # %d Dollar Gain/Loss $%8.2f' % (date, self.trades, self.units*(self.sell_price-self.buy_price)))
            self.cum_gain += self.units*(self.sell_price-self.buy_price)
            print('%s | Trade # %d Cummulative Gain/Loss $%8.2f' % (date, self.trades,self.cum_gain))

    def close_out(self, bar, position):
        date, price = self.get_date_price(bar)
        #print(50 * '=')
        if position == 1:
          #self.sell_price = price*(1 - self.ptc)
          #self.amount += self.units * (price * (1 - self.ptc))
          self.amount += self.units * self.buy_price
          #self.trades += 1
          self.trades -= 1
          #self.win_count += np.where(self.buy_price < price, 1, 0)
          print("%s | Trade # %d Not enough data to complete trade. Excluding This LONG Trade From Final Results" % (date, self.trades))
          #print("%s | Trade # %d Sell Price w/o TCs = $%8.2f"% (date, self.trades,price))
          #print("%s | Trade # %d Sell Price w/ TCs = $%8.2f"% (date, self.trades,self.sell_price))
          print("%s | Trade # %d # of Units = %8.2f"% (date, self.trades,self.units))
          #print('%s | Trade # %d Gain/Loss %8.2f%%' % (date, self.trades, ((self.sell_price-self.buy_price)/self.buy_price)*100))
          #print('%s | Trade # %d Dollar Gain/Loss $%8.2f' % (date, self.trades, self.units*(self.sell_price-self.buy_price)))
          #self.cum_gain += self.units*(self.sell_price-self.buy_price)
          print('%s | Trade # %d Cummulative Gain/Loss $%8.2f' % (date, self.trades,self.cum_gain))
        elif position == -1:
          #self.buy_price = price*(1 + self.ptc)
          #self.amount -= self.units * (price * (1 + self.ptc))
          self.amount -= self.units * self.sell_price
          #self.trades += 1
          self.trades -= 1
          #self.win_count += np.where(self.sell_price > price, 1, 0)
          print("%s | Trade # %d Not enought data to complete trade. Excluding This SHORT Trade From Final Results" % (date, self.trades))
          #print("%s | Trade # %d Buy Price w/o TCs = $%8.2f"% (date, self.trades,price))
          #print("%s | Trade # %d Buy Price w/ TCs = $%8.2f"% (date, self.trades,self.buy_price))
          print("%s | Trade # %d # of Units = %8.2f"% (date, self.trades,self.units))
          #print('%s | Trade # %d Gain/Loss %8.2f%%' % (date, self.trades, ((self.sell_price-(self.buy_price))/self.sell_price)*100))
          #print('%s | Trade # %d Dollar Gain/Loss $%8.2f' % (date, self.trades, self.units*(self.sell_price-self.buy_price)))
          #self.cum_gain += self.units*(self.sell_price-self.buy_price)
          print('%s | Trade # %d Cummulative Gain/Loss $%8.2f' % (date, self.trades,self.cum_gain))

        print("%s | Trade # %d STARTING AMOUNT = %8.2f" % (date, self.trades,self.initial_amount))
        print("%s | Trade # %d FINAL AMOUNT = %8.2f" % (date, self.trades,self.amount))
        perf = (self.amount / self.initial_amount - 1) * 100
        print('%s | NET RETURN OF STRATEGY = %7.2f%%' % (date, perf))
        print("Number of Indivdual Trades: ",self.trades)
        self.rt_trades=(self.trades/2)
        print("Number of Round Trip Trades: ",self.rt_trades)
        print('Number of Winning Trades:',self.win_count)
        if self.trades >= 1:
          self.win_rate = (self.win_count/(self.rt_trades))*100
        else:
          self.win_rate = 0
        print('Win Rate: %7.2f%%' % self.win_rate)
        #print(110 * '=')
        return perf, self.rt_trades, self.win_rate

In [11]:
class BacktestingModelsStrategy(BacktestingBase):
    #def __init__(self, symbol, amount, algo_type, SMA1, SMA2, EMA1, EMA2, Vol1, Vol2, L1, L2, MaxD, MSL, Win):
    def __init__(self, symbol, amount, ptc, algo_type, Win):
        super(BacktestingModelsStrategy, self).__init__(symbol, amount, ptc)
        self.symbol = symbol
        self.algo_type = algo_type
        self.lags = 5
        self.L1 = 1.5
        #self.L1 = L1
        self.L2 = 8
        #self.L2 = L2
        self.SMA1 = 35
        #self.SMA1 = SMA1
        self.SMA2 = 196
        #self.SMA2 = SMA2
        self.EMA1 = 50
        #self.EMA1 = EMA1
        self.EMA2 = 180
        #self.EMA2 = EMA2
        self.Vol1 = 90
        #self.Vol1 = Vol1
        self.Vol2 = 190
        #self.Vol2 = Vol2
        self.Win = Win
        #self.MaxD = MaxD
        #self.MSL = MSL
        self.cols = []
        self.df = self.data.copy()
        self.prepare_statistics()
        self.results = pd.DataFrame()

    def prepare_statistics(self):
        #self.df = self.df.dropna(inplace=True)
        self.df['returns'] = np.log(self.df[self.symbol] / self.df[self.symbol].shift(1))

        self.lr_sigma = self.df['returns'].std()
        self.df['d_cat'] = np.digitize(self.df['returns'], bins=[-1*self.L2*self.lr_sigma, -1*self.L1*self.lr_sigma, self.L1*self.lr_sigma, self.L2*self.lr_sigma])

        self.df['SMA1'] = self.df[self.symbol].rolling(self.SMA1).mean()
        self.df['SMA2'] = self.df[self.symbol].rolling(self.SMA2).mean()
        self.df['SMA_Diff'] = self.df['SMA1']-self.df['SMA2']

        self.df['EMA1'] = self.df[self.symbol].ewm(span=self.EMA1, adjust=False).mean()
        self.df['EMA2'] = self.df[self.symbol].ewm(span=self.EMA2, adjust=False).mean()
        self.df['EMA_Diff'] = self.df['EMA1']-self.df['EMA2']

        self.df['Vol1'] = self.df['returns'].rolling(self.Vol1).std()
        self.df['Vol2'] = self.df['returns'].rolling(self.Vol2).std()
        #self.df.dropna(inplace=True)
        self.df['d'] = np.where(self.df['returns'] > 0, 1, 0)

        self.features = ['returns', 'd', 'd_cat', 'SMA1', 'SMA2', 'SMA_Diff','EMA1', 'EMA2', 'EMA_Diff', 'Vol1', 'Vol2']

        for f in self.features:
            for lag in range(1, self.lags + 1):
                col = f'{f}_lag_{lag}'
                self.df[col] = self.df[f].shift(lag)
                self.cols.append(col)
        self.df.dropna(inplace=True)
        #print("Return df and cols")
        #print("Length of df= ",len(self.df))
        return self.df, self.cols

    def normalize_data(self, data):
        split = int(len(self.data) * 0.7)
        #print("split = ",split)
        self.train = self.data.iloc[:split].copy()
        self.test = self.data.iloc[split:].copy()
        #print("Training Length = ",len(self.train))
        #print("Testing Length = ",len(self.test))
        #print("values in self.train = ",self.train)
        mu, std = self.train.mean(), self.train.std()
        self.train_ = (self.train - mu) / std
        self.test_ = (self.test-mu)/std
        #print("Normalized Training Length = ",len(self.train_))
        #print("Normalized_Testing Length = ",len(self.test_))
        return self.train_,self.test_,self.train,self.test
        #return self.train,self.test

    def run_strategy(self, Parms=None):
        self.position = 0
        self.final_bar = 0
        self.units = 0
        self.trades = 0
        self.count_scenarios = 0
        self.amount = self.initial_amount
        if Parms is not None:
            #self.SMA1 = Parms[0]
            #print("SMA1 = ", self.SMA1)
            #self.SMA2 = Parms[1]
            #print("SMA2 = ", self.SMA2)
            #self.EMA1 = Parms[2]
            #print("EMA1 = ", self.EMA1)
            #self.EMA2 = Parms[3]
            #print("EMA2 = ", self.EMA2)
            #self.Vol1 = Parms[4]
            #print("Vol1 = ", self.Vol1)
            #self.Vol2 = Parms[5]
            #print("Vol2 = ", self.Vol2)
            #self.L1 = Parms[6]
            #print("L1 = ", self.L1)
            #self.L2 = Parms[7]
            #print("L2 = ", self.L2)
            #self.MaxD = Parms[8]
            #self.MaxD = Parms[0]
            #print("MaxD = ", self.MaxD)
            #self.MSL = Parms[9]
            #self.MSL = Parms[1]
            #print("MSL = ", self.MSL)
            #self.Win = Parms[10]
            #self.Win = Parms[0]
            print("Win = ", self.Win)
            #print('Calling method prepare_statistics')
            self.data, self.cols = self.prepare_statistics()

        #print('Splitting & Normalizing Data')
        self.train_,self.test_,self.train,self.test = self.normalize_data(self.data)

        print(110 * '=')
        print(f'*** START BACKTESTING ***')
        print(f'SYM={self.symbol} | Lags={self.lags} | SMA1={self.SMA1} | SMA2={self.SMA2} | L1={self.L1}| L2={self.L2} | EMA1={self.EMA1} | EMA2={self.EMA2} | Vol1={self.Vol1}| Vol2={self.Vol2} | Win={self.Win}')
        print(110 * '=')

        if self.algo_type == "DTC":
          print('Creating DTC Model')
          #self.model = tree.DecisionTreeClassifier(max_depth=3,min_samples_leaf=5)
          #self.model = tree.DecisionTreeClassifier(max_depth=2,min_samples_leaf=25)
          self.model = tree.DecisionTreeClassifier(max_depth=1,min_samples_leaf=20)
          #self.model = tree.DecisionTreeClassifier(max_depth=self.MaxD,min_samples_leaf=self.MSL)
        elif self.algo_type == "Gaussian":
          print('Creating GaussianNB Model')
          self.model = GaussianNB()
        elif self.algo_type == "LR":
          print('Creating LogisticRegression Model')
          #self.model = LogisticRegression(C=0.001, solver='saga', multi_class='auto', max_iter=3100)
          #self.model = LogisticRegression(C=0.1, solver='lbfgs', multi_class='auto', max_iter=3100)
          self.model = LogisticRegression(C=0.00001, solver='saga', max_iter=3100)
        elif self.algo_type == "SVC":
          print('Creating SVC Model')
          self.model = SVC(C=0.1, kernel='rbf', probability=True)
        elif self.algo_type == "MLPC":
          print('Creating MLPClassifier Model')
          self.model = MLPClassifier(hidden_layer_sizes=[48], max_iter=2000, random_state=100,shuffle=False)
        else:
          print('Please enter a valid model type: DTC, Gaussian, LR, SVC, MLPC')

        #n is the sliding window width/size
        n = self.Win
        start = n

        #for bar in range(len(self.test)):
        for bar in range(start, len(self.test)):
              #print(f'Fitting {self.algo_type} Model')
              self.model.fit(self.train_[self.cols].iloc[bar-n:bar], self.train['d'].iloc[bar-n:bar])
              #print('Perform Prediction on Normalized Training Data (train_)')
              self.train.loc[self.train.index[bar], 'p'] = self.model.predict(self.train_[self.cols].iloc[bar-n:bar])[0]
              #print('Perform Prediction on Normalized Test Data (test_)')
              self.test.loc[self.test.index[bar],'p'] = self.model.predict(self.test_[self.cols].iloc[bar-n:bar])[0]
              if self.position == 0 or self.position ==-1:
                  if self.test['p'].iloc[bar] > 0:
                      if self.position == -1:
                        self.place_buy_order(bar, self.position)
                        self.position = 0
                      self.place_buy_order(bar, self.position)
                      self.position = 1
                      #print(55*'=')
              if self.position == 0 or self.position == 1:
                  if self.test['p'].iloc[bar] <= 0:
                      if self.position == 1:
                        self.place_sell_order(bar, self.position)
                        self.position = 0
                      self.place_sell_order(bar, self.position)
                      self.position = -1
                      #print(55*'=')
              self.final_bar = bar
        perf, trades, win_rate = self.close_out(self.final_bar, self.position)
        self.train.dropna(inplace=True)
        accuracy_is = accuracy_score(self.train['d'], self.train['p'])
        print("IN SAMPLE Model Prediction Accuracy train['d'] vs train['p'] (%) = ", accuracy_is*100)
        p_nans_test = self.test['p'].isnull().sum()
        accuracy_oos = accuracy_score(self.test['d'].iloc[p_nans_test:], self.test['p'].iloc[p_nans_test:])
        print("OUT OF SAMPLE Model Prediction Accuracy test['d'] vs test['p'] (%) = ", accuracy_oos*100)
        #print("Value of final_bar = ",self.final_bar)
        self.results[['Perf(%)','Trades','WinRate','AIS', 'AOOS','Lags','SMA1','SMA2','L1','L2','EMA1','EMA2','Vol1','Vol2','Win']] = [[perf, trades, win_rate, accuracy_is, accuracy_oos, self.lags, self.SMA1, self.SMA2, self.L1, self.L2, self.EMA1, self.EMA2, self.Vol1, self.Vol2, self.Win]]
        print(110 * '=')
        return self.results




In [14]:
def DTC_OOS():
  out_of_sample_results = pd.DataFrame()
  #Set proportional transaction cost at 0.5%
  ptc = 0.005
  #Sliding window size set to 385 days
  Win = 385
  DTC = BacktestingModelsStrategy('AAPL.O', 10000, ptc, 'DTC', Win)
  out_of_sample_results = DTC.run_strategy(Win)

In [15]:
%%time
DTC_OOS_Results = DTC_OOS()

Win =  385
*** START BACKTESTING ***
SYM=AAPL.O | Lags=5 | SMA1=35 | SMA2=196 | L1=1.5| L2=8 | EMA1=50 | EMA2=180 | Vol1=90| Vol2=190 | Win=385
Creating DTC Model
2018-08-27 | Trade # 1 Opening LONG Trade
2018-08-27 | Trade # 1 Amount = $10000.00
2018-08-27 | Trade # 1 Buy Price w/o TCs = $  217.94
2018-08-27 | Trade # 1 Buy Price w/ TCs = $  219.03
2018-08-27 | Trade # 1 # of Units =    45.00
2018-09-26 | Trade # 2 Closing LONG Trade
2018-09-26 | Trade # 2 Sell Price w/o TCs = $  220.42
2018-09-26 | Trade # 2 Sell Price w/ TCs = $  219.32
2018-09-26 | Trade # 2 # of Units =    45.00
2018-09-26 | Trade # 2 Amount = 10012.97
2018-09-26 | Trade # 2 Gain/Loss     0.13%
2018-09-26 | Trade # 2 Dollar Gain/Loss $   12.97
2018-09-26 | Trade # 2 Cummulative Gain/Loss $   12.97
2018-09-26 | Trade # 3 Opening SHORT Trade
2018-09-26 | Trade # 3 Amount = $10012.97
2018-09-26 | Trade # 3 Sell Price w/o TCs = $  220.42
2018-09-26 | Trade # 3 Sell Price w/ TCs = $  219.32
2018-09-26 | Trade # 3 # of 

In [16]:
def Gaussian_OOS():
  out_of_sample_results = pd.DataFrame()
  #Set proportional transaction cost at 0.5%
  ptc = 0.005
  #Sliding window size set to 405 days
  Win = 405
  Gaussian = BacktestingModelsStrategy('AAPL.O', 10000, ptc, 'Gaussian', Win)
  out_of_sample_results = Gaussian.run_strategy(Win)

In [17]:
%%time
Gaussian_OOS_Results = Gaussian_OOS()

Win =  405
*** START BACKTESTING ***
SYM=AAPL.O | Lags=5 | SMA1=35 | SMA2=196 | L1=1.5| L2=8 | EMA1=50 | EMA2=180 | Vol1=90| Vol2=190 | Win=405
Creating GaussianNB Model
2018-09-25 | Trade # 1 Opening SHORT Trade
2018-09-25 | Trade # 1 Amount = $10000.00
2018-09-25 | Trade # 1 Sell Price w/o TCs = $  222.19
2018-09-25 | Trade # 1 Sell Price w/ TCs = $  221.08
2018-09-25 | Trade # 1 # of Units =    44.00
2019-01-04 | Trade # 2 Closing SHORT Trade
2019-01-04 | Trade # 2 Buy Price w/o TCs = $  148.26
2019-01-04 | Trade # 2 Buy Price w/ TCs = $  149.00
2019-01-04 | Trade # 2 # of Units =    44.00
2019-01-04 | Trade # 2 Amount = 13171.42
2019-01-04 | Trade # 2 Gain/Loss    32.60%
2019-01-04 | Trade # 2 Dollar Gain/Loss $ 3171.42
2019-01-04 | Trade # 2 Cummulative Gain/Loss $ 3171.42
2019-01-04 | Trade # 3 Opening LONG Trade
2019-01-04 | Trade # 3 Amount = $13171.42
2019-01-04 | Trade # 3 Buy Price w/o TCs = $  148.26
2019-01-04 | Trade # 3 Buy Price w/ TCs = $  149.00
2019-01-04 | Trade # 3

In [18]:
def LR_OOS():
  out_of_sample_results = pd.DataFrame()
  #Set proportional transaction cost at 0.5%
  ptc = 0.005
  #Sliding window size set to 300 days
  Win = 300
  LR = BacktestingModelsStrategy('AAPL.O', 10000, ptc, 'LR', 300)
  out_of_sample_results = LR.run_strategy(Win)

In [19]:
%%time
LR_OOS_Results = LR_OOS()

Win =  300
*** START BACKTESTING ***
SYM=AAPL.O | Lags=5 | SMA1=35 | SMA2=196 | L1=1.5| L2=8 | EMA1=50 | EMA2=180 | Vol1=90| Vol2=190 | Win=300
Creating LogisticRegression Model
2018-04-26 | Trade # 1 Opening LONG Trade
2018-04-26 | Trade # 1 Amount = $10000.00
2018-04-26 | Trade # 1 Buy Price w/o TCs = $  164.22
2018-04-26 | Trade # 1 Buy Price w/ TCs = $  165.04
2018-04-26 | Trade # 1 # of Units =    60.00
2018-09-12 | Trade # 2 Closing LONG Trade
2018-09-12 | Trade # 2 Sell Price w/o TCs = $  221.07
2018-09-12 | Trade # 2 Sell Price w/ TCs = $  219.96
2018-09-12 | Trade # 2 # of Units =    60.00
2018-09-12 | Trade # 2 Amount = 13295.41
2018-09-12 | Trade # 2 Gain/Loss    33.28%
2018-09-12 | Trade # 2 Dollar Gain/Loss $ 3295.41
2018-09-12 | Trade # 2 Cummulative Gain/Loss $ 3295.41
2018-09-12 | Trade # 3 Opening SHORT Trade
2018-09-12 | Trade # 3 Amount = $13295.41
2018-09-12 | Trade # 3 Sell Price w/o TCs = $  221.07
2018-09-12 | Trade # 3 Sell Price w/ TCs = $  219.96
2018-09-12 | 

In [22]:
def SVC_OOS():
  out_of_sample_results = pd.DataFrame()
  #Set proportional transaction cost at 0.5%
  ptc = 0.005
  #Sliding window size set to 60 days
  Win = 60
  SVC = BacktestingModelsStrategy('AAPL.O', 10000, ptc, 'SVC', Win)
  out_of_sample_results = SVC.run_strategy(Win)

In [23]:
%%time
SVC_OOS_Results = SVC_OOS()

Win =  60
*** START BACKTESTING ***
SYM=AAPL.O | Lags=5 | SMA1=35 | SMA2=196 | L1=1.5| L2=8 | EMA1=50 | EMA2=180 | Vol1=90| Vol2=190 | Win=60
Creating SVC Model
2017-05-12 | Trade # 1 Opening LONG Trade
2017-05-12 | Trade # 1 Amount = $10000.00
2017-05-12 | Trade # 1 Buy Price w/o TCs = $  156.10
2017-05-12 | Trade # 1 Buy Price w/ TCs = $  156.88
2017-05-12 | Trade # 1 # of Units =    63.00
2017-06-02 | Trade # 2 Closing LONG Trade
2017-06-02 | Trade # 2 Sell Price w/o TCs = $  155.45
2017-06-02 | Trade # 2 Sell Price w/ TCs = $  154.67
2017-06-02 | Trade # 2 # of Units =    63.00
2017-06-02 | Trade # 2 Amount =  9860.91
2017-06-02 | Trade # 2 Gain/Loss    -1.41%
2017-06-02 | Trade # 2 Dollar Gain/Loss $ -139.09
2017-06-02 | Trade # 2 Cummulative Gain/Loss $ -139.09
2017-06-02 | Trade # 3 Opening SHORT Trade
2017-06-02 | Trade # 3 Amount = $ 9860.91
2017-06-02 | Trade # 3 Sell Price w/o TCs = $  155.45
2017-06-02 | Trade # 3 Sell Price w/ TCs = $  154.67
2017-06-02 | Trade # 3 # of Un

In [24]:
def MLPC_OOS():
  out_of_sample_results = pd.DataFrame()
  #Set proportional transaction cost at 0.5%
  ptc = 0.005
  #Sliding window size set to 360 days
  Win = 360
  MLPC = BacktestingModelsStrategy('AAPL.O', 10000, ptc, 'MLPC', Win)
  out_of_sample_results = MLPC.run_strategy(Win)

In [25]:
%%time
MLPC_OOS_Results = MLPC_OOS()

Win =  360
*** START BACKTESTING ***
SYM=AAPL.O | Lags=5 | SMA1=35 | SMA2=196 | L1=1.5| L2=8 | EMA1=50 | EMA2=180 | Vol1=90| Vol2=190 | Win=360
Creating MLPClassifier Model
2018-07-23 | Trade # 1 Opening LONG Trade
2018-07-23 | Trade # 1 Amount = $10000.00
2018-07-23 | Trade # 1 Buy Price w/o TCs = $  191.61
2018-07-23 | Trade # 1 Buy Price w/ TCs = $  192.57
2018-07-23 | Trade # 1 # of Units =    51.00
2019-04-24 | Trade # 2 Closing LONG Trade
2019-04-24 | Trade # 2 Sell Price w/o TCs = $  207.16
2019-04-24 | Trade # 2 Sell Price w/ TCs = $  206.12
2019-04-24 | Trade # 2 # of Units =    51.00
2019-04-24 | Trade # 2 Amount = 10691.36
2019-04-24 | Trade # 2 Gain/Loss     7.04%
2019-04-24 | Trade # 2 Dollar Gain/Loss $  691.36
2019-04-24 | Trade # 2 Cummulative Gain/Loss $  691.36
2019-04-24 | Trade # 3 Opening SHORT Trade
2019-04-24 | Trade # 3 Amount = $10691.36
2019-04-24 | Trade # 3 Sell Price w/o TCs = $  207.16
2019-04-24 | Trade # 3 Sell Price w/ TCs = $  206.12
2019-04-24 | Trade